In [50]:
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd
from datetime import datetime

# show all columns
pd.options.display.max_columns = None

In [51]:
# create output directory
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = '.\\Outputs'

In [52]:
# connect to AGOL
gis = GIS()

In [53]:
# get data from AGOL
tip_lines = gis.content.get('3dd79d21ce4342c0a2f2f8ccd4e12df2').layers[0].query(where='1=1').sdf
tip_points = gis.content.get('3dd79d21ce4342c0a2f2f8ccd4e12df2').layers[1].query(where='1=1').sdf
tip_comments = gis.content.get('3dd79d21ce4342c0a2f2f8ccd4e12df2').tables[0].query(where='1=1').sdf

In [54]:
# remove fields
del tip_comments['GlobalID']
del tip_comments['OBJECTID']

In [55]:
# join comments to lines
tip_comments_on_lines = tip_comments.merge(tip_lines[['GlobalID', 'PIN', 'PIN_DESC']], left_on='GUID', right_on='GlobalID', how='inner')

In [56]:
# joins comments to points
tip_comments_on_points = tip_comments.merge(tip_points[['GlobalID', 'PIN', 'PIN_DESC']], left_on='GUID', right_on='GlobalID', how='inner')

In [57]:
tip_comments_on_all = pd.concat([tip_comments_on_points, tip_comments_on_lines])
tip_comments_on_all.head()

,PersonName,PersonOrg,PersonCont,Comment,CommentDT,GUID,GlobalID,PIN,PIN_DESC
0,Frederick Michel Jenny,Citizen,frederickmjenny@gmail.com,I want to voice my opposition to any EV chargi...,2024-07-02 21:08:10.910000,95eff3b3-c54d-4216-8ecb-842d5c338e44,95eff3b3-c54d-4216-8ecb-842d5c338e44,22097.0,Riverton City Electric Vehicle Charging Stations
1,Frederick Michel Jenny,Citizen,frederickmjenny@gmail.com,The S Line is amazing and should be expanded a...,2024-07-02 21:51:25.956999,d90265e7-c9ef-4466-920b-c0305b60d9df,d90265e7-c9ef-4466-920b-c0305b60d9df,21826.0,S-Line Extension
2,CAROL HANSEN,,chansen@weber.edu,"Yes, let’s keep extending the SLine on the Eas...",2024-07-03 14:02:47.008000,d90265e7-c9ef-4466-920b-c0305b60d9df,d90265e7-c9ef-4466-920b-c0305b60d9df,21826.0,S-Line Extension
3,Cody,Resident SH,2088910994,"I love the S-Line! Use it often, should be ext...",2024-07-05 06:36:45.075000,d90265e7-c9ef-4466-920b-c0305b60d9df,d90265e7-c9ef-4466-920b-c0305b60d9df,21826.0,S-Line Extension
4,Frederick Michel Jenny,Citizen,frederickmjenny@gmail.com,Adding another frontrunner station down here w...,2024-07-02 23:40:47.760000,1278918e-685c-4668-a384-60857a579fbf,1278918e-685c-4668-a384-60857a579fbf,21213.0,FrontRunner Station at Point of the Mountain


In [58]:
# export with date
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")
tip_comments_on_all.to_csv(os.path.join(outputs, f'tip_comments_{dt_string}.csv'), index=False)